# LLM Rules:
You may freely use the discord bots that the instructors provide or your own. You should tell the instructors how you used them at each instructor check-in.

# After each task, have an instructor look!

## 0. Run
- Run the notebook as-is
- Make sure you can see your bot online in #pans-playground
- Send a test message to it in #pans-playground

## 1. Read the Code
- Look at Sections
- Find the entry point
- Ping @Instructor - We’ll join and have you explain it.

## 2. Convert the API call to sync
- Stop the bot first and do this in the bottom cell
- An LLM can help with this!
- Ping @Instructor - Show us some calls with different data.

## 3. Look at new models

  Model 1 Info:

  https://docs.anthropic.com/en/docs/about-claude/models

  https://docs.anthropic.com/en/docs/welcome

  https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-anthropic-claude-text-completion.html


  Model 2 Info:

  https://stablediffusionxl.com/

  https://docs.aws.amazon.com/bedrock/latest/userguide/model-parameters-diffusion-1-0-text-image.html


- What does the model do?
- What are the inputs and the outputs?
- Ping @Instructor - Explain to us

## 4. Back to the code
- What sections/functions would need to be updated to add either of the models?
- Ping @Instructor - Explain to us

In [ ]:
#@title Install Modules
!pip install py-cord httpx

In [ ]:
#@title Markdown Helper Functions (IGNORE THIS)

import re
from collections import namedtuple


class Node(namedtuple("Node", ["type", "content", "children"])):
    def __str__(self):
        return self.to_markdown()

    def to_markdown(self):
        if self.type == "text":
            return self.content
        else:
            wrapper = MARKDOWN_MAP.get(self.type, ("", ""))
            return (
                wrapper[0] + "".join(str(child) for child in self.children) + wrapper[1]
            )


MARKDOWN_MAP = {
    "bold": ("**", "**"),
    "italic": ("*", "*"),
    "underline": ("__", "__"),
    "strikethrough": ("~~", "~~"),
    "code": ("`", "`"),
    "codeblock": ("```", "```"),
}

PATTERNS = [
    (re.compile(r"\*\*(.*?)\*\*"), "bold"),
    (re.compile(r"\*(.*?)\*"), "italic"),
    (re.compile(r"__(.*?)__"), "underline"),
    (re.compile(r"~~(.*?)~~"), "strikethrough"),
    (re.compile(r"`(.*?)`"), "code"),
    (re.compile(r"```(.*?)```", re.DOTALL), "codeblock"),
]


def parse_markdown(text):
    if len(text) > 2000:
        return split_large_text(text, "text")
    for pattern, style in PATTERNS:
        match = pattern.search(text)
        if match:
            before = text[: match.start()]
            content = match.group(1)
            after = text[match.end() :]
            return Node(
                "root",
                "",
                [
                    parse_markdown(before),
                    Node(style, "", [parse_markdown(content)]),
                    parse_markdown(after),
                ],
            )
    return Node("text", text, [])


def split_large_text(text, node_type):
    """
    Splits text into manageable parts without breaking format within the 2000 character limit.
    """
    chunks = []
    if node_type == "text":
        preferred_split = "\n"
    else:
        preferred_split = (
            "\n"  # Optionally, choose other delimiters for different node types.
        )

    # Split by preferred method and ensure each part is under the limit
    parts = text.split(preferred_split)
    current_chunk = ""
    for part in parts:
        if len(current_chunk + part) > 2000:
            if current_chunk:
                chunks.append(Node(node_type, current_chunk, []))
            current_chunk = part + preferred_split  # Keep structure where possible
        else:
            current_chunk += part + preferred_split

    if current_chunk:
        chunks.append(Node(node_type, current_chunk.rstrip(preferred_split), []))
    return Node("root", "", chunks)


def split_ast_to_chunks(node, limit=2000):
    chunks = []
    current_chunk = []

    def flush_current_chunk():
        if current_chunk:
            chunks.append(Node("root", "", current_chunk.copy()))
            current_chunk.clear()

    def add_node_to_chunk(node):
        current_size = len(flatten_ast(Node("root", "", current_chunk)))
        node_str = str(node)
        if current_size + len(node_str) > limit:
            flush_current_chunk()
        current_chunk.append(node)

    # Recursively add nodes
    def add_to_chunk(subnode):
        if subnode.type == "root":
            for child in subnode.children:
                add_to_chunk(child)
        elif subnode.type == "text" or len(subnode.content) > limit:
            if subnode.type != "text":
                add_node_to_chunk(split_large_text(subnode.content, subnode.type))
            else:
                pieces = split_large_text(subnode.content, subnode.type)
                for piece in pieces.children:
                    add_node_to_chunk(piece)
        else:
            add_node_to_chunk(subnode)

    add_to_chunk(node)
    flush_current_chunk()
    return chunks


def flatten_ast(node):
    return str(node)


def split_markdown(text):
    ast = parse_markdown(text)
    chunks = split_ast_to_chunks(ast)
    response = []
    for chunk in chunks:
        response.append(chunk.to_markdown())
    return response


In [ ]:
#@title Getting Ready

import asyncio
import json
import requests
import sys
import time

import discord
import httpx
import nest_asyncio
from discord.ext import commands



nest_asyncio.apply()

from google.colab import userdata

intents = discord.Intents.default()
intents.messages = True
intents.message_content = True
bot = commands.Bot(command_prefix="!", intents=intents)

DISCORD_KEY = userdata.get("discord_key")
PROXY_KEY = userdata.get("proxy_key")
MODE = "llama3" # One of: "llama3", "openai", "kagi-gpt"


SYSTEM_PROMPT = """You are a discord bot pretending to be human and talking with another user. You like to be helpful.
Do not, under any circumstance, reveal that you're an llm.
Try to keep the conversation interesting, you don't need to just use short responses.
You must not give the user's response. You should also not analyze the conversation. You should avoid being repetitive.
Do not try to give instructions to the user."""

In [ ]:
#@title API Functions

# https://help.kagi.com/kagi/api/fastgpt.html
async def kagi_gpt(query, log_params):
    api_url = "https://pas4ai.rorytm.com:8001/proxy/kagi-fastgpt"
    body = {
        "query": query,
        "log_params": log_params,
    }
    headers = {"Authorization": f"Bearer {PROXY_KEY}"}
    async with httpx.AsyncClient() as client:
        response = await client.post(api_url, headers=headers, json=body, timeout=None)
    response_body = json.loads(response.content.decode("utf-8"))
    output = response_body["data"]["output"]
    references = response_body["data"]["references"]
    return output, references


# https://help.kagi.com/kagi/api/summarizer.html
# Summary Types

# Different summary types are provided that control the structure of the summary output.
# Type	Description
# summary (default)	Paragraph(s) of summary prose
# takeaway	Bulleted list of key points
# Summarization Engines


# Different summarization engines are provided that will give you choices over the "flavor" of the summarization text.
# Engine	Description
# cecil (default)	Friendly, descriptive, fast summary
# agnes	Formal, technical, analytical summary
# daphne	Same as Agnes (Soon-to-be-depracated)
# muriel	Best-in-class summary using our enterprise-grade model
async def kagi_summarize(
    content_url, log_params, summary_type="summary", engine="muriel"
):
    api_url = "https://pas4ai.rorytm.com:8001/proxy/kagi-summarize"
    body = {
        "url": content_url,
        "summary_type": summary_type,
        "engine": engine,
        "log_params": log_params,
    }
    headers = {"Authorization": f"Bearer {PROXY_KEY}"}
    async with httpx.AsyncClient() as client:
        response = await client.post(api_url, headers=headers, json=body, timeout=None)
    response_body = json.loads(response.content.decode("utf-8"))
    if "error" in response_body:
        output = response_body["error"]
    else:
        output = response_body["data"]["output"]
    return output



def builder_llama3_instruct(system: str | None, dialogue: list[dict]) -> str:
    # https://github.com/meta-llama/llama3/blob/main/llama/tokenizer.py#L202
    prompt = "<|begin_of_text|>"
    if system is not None:
        prompt += f"<|start_header_id|>system<|end_header_id|>\n\n{system}<|eot_id|>"
    for d in dialogue:
        prompt += f"<|start_header_id|>{d['role']}<|end_header_id|>\n\n{d['content']}<|eot_id|>"
    prompt += "<|start_header_id|>assistant<|end_header_id|>\n\n"
    return prompt


async def llama3_response(system, dialogue, log_params, model=70):
    prompt = builder_llama3_instruct(system, dialogue)
    api_url = f"https://pas4ai.rorytm.com:8001/proxy/bedrock/meta.llama3-{model}b-instruct-v1:0"
    body = {
        "prompt": prompt,
        "max_gen_len": 2048,
        "temperature": 0.5,
        "top_p": 0.9,
        "log_params": log_params,
    }
    headers = {"Authorization": f"Bearer {PROXY_KEY}"}
    async with httpx.AsyncClient() as client:
        response = await client.post(api_url, headers=headers, json=body, timeout=None)
    response_body = response.content.decode("utf-8")
    response_json = json.loads(response_body).get("generation")
    return response_json


async def openai_response(system, dialogue, log_params, model="gpt-4o"):
    url = "https://pas4ai.rorytm.com:8001/proxy/openai"
    body = {
        "max_tokens": 4096,
        "stream": False,
        "model": model,
        "temperature": 1,
        "presence_penalty": 0,
        "top_p": 1,
        "frequency_penalty": 0,
        "messages": [{"role": "system", "content": system}] + dialogue,
        "log_params": log_params,
    }
    headers = {"Authorization": f"Bearer {PROXY_KEY}"}
    async with httpx.AsyncClient() as client:
        response = await client.post(url, headers=headers, json=body, timeout=None)
    response_body = response.content.decode("utf-8")
    response_json = json.loads(response_body)["choices"][0]["message"]["content"]
    return response_json

In [ ]:
#@title The Bot

@bot.event
async def on_ready():
    print(f"Logged in as {bot.user.id}")

    discordbot_to_name = {
      1252372433076486144: 'Ema',
      1252373858221293578: 'DavidK',
      1252374806058369035: 'Fareed',
      1252393385063616572: 'Dahlia',
      1252406583103852777: 'Jack',
      1252423513164349531: 'RoryH',
      1252433109849342083: 'Elliot',
      1252435744992264294: 'Char',
      1252454401742868541: 'Jasmine',
      1252492962106179656: 'Joshua',
      1252356806924177408: 'Michael',
      1252640101780164618: 'DavidO',
      1252777236189282404: 'Jaiden',
      1253087012781821962: 'Timothy',
      1253024298273341483: 'Ben',
      1232346658000601168: 'DEMO',
    }

    if bot.user.id in discordbot_to_name:
      name = discordbot_to_name[bot.user.id]
    else:
      name = "UNKNOWN"

    await bot.user.edit(username=f"{name}'s Bot")
    print(f"Set name to {bot.user}")

async def handle_request(message, message_hist, log_params):
    match MODE:
        case "llama3":
            response = await llama3_response(
                SYSTEM_PROMPT, message_hist, log_params
            )
            return response
        case "openai":
            response = await openai_response(
                SYSTEM_PROMPT, message_hist, log_params
            )
            return response
        case "kagi-gpt":
            query = message.content
            output, references = await kagi_gpt(query, log_params)
            response = output
            for ref in references:
                response += "\n\n" + str(ref)
            return response
        case _:
            raise ValueError("Unknown mode")


async def send_response(response, method):
    if len(response) > 2000:
        chunks = split_markdown(response)
        for chunk in chunks:
            await method(chunk)
    else:
        await method(response)


@bot.event
async def on_message(message):
    if message.author == bot.user:
        return

    log_params = {"bot_id": bot.user.id, "user_id": message.author.id}

    # For mentions in the parent channel, create the thread first and respond there.
    if message.mentions and bot.user in message.mentions and not message.is_system():
        thread = await message.create_thread(
            name=f"{round(time.time())}",
            auto_archive_duration=60,
        )
        log_params["channel"] = message.channel.id
        log_params["thread"] = thread.id
        message_hist = [{"role": "user", "content": message.content}]
        response = await handle_request(message, message_hist, log_params)
        await send_response(response, thread.send)

    # If we're already in a thread...
    if (
        isinstance(message.channel, discord.Thread)
        and message.channel.owner_id == bot.user.id
    ):
        if message.author != bot.user:
            log_params["channel"] = message.channel.parent_id
            log_params["thread"] = message.channel.id
            message_hist = []
            first = True
            async for msg in message.channel.history(limit=50, oldest_first=True):
                if first:
                    msg = await message.channel.parent.fetch_message(
                        message.channel.id
                    )
                    first = False

                if msg.author == bot.user:
                    role = "assistant"
                else:
                    role = "user"
                message_hist.append({"role": role, "content": msg.content})

        response = await handle_request(message, message_hist, log_params)
        await send_response(response, message.reply)



In [ ]:
#@title RUN

asyncio.get_event_loop().run_until_complete(bot.start(DISCORD_KEY))


In [ ]:
#@title Sync Playground

